In [1]:
import os

In [2]:
%pwd

'c:\\Users\\lenov\\OneDrive\\Documents\\ML_MLOps\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\lenov\\OneDrive\\Documents\\ML_MLOps'

In [6]:
import dagshub
dagshub.init(repo_owner='mjdocevedo', repo_name='ML_MLOps', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=b58f36ef-baf8-4a36-8a33-a4acb490ac99&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=d14a93af3c39723e4dc4b39bd3a999243b9c1107a8f1445b2c9ab0a91c9b921d




c:\Users\lenov\OneDrive\Documents\ML_MLOps\ml_mlops_env\Lib\site-packages\rich\live.py:229: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Initialized MLflow to track repo "mjdocevedo/ML_MLOps"

Repository mjdocevedo/ML_MLOps initialized!

In [21]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_X_path: Path
    test_y_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    metric_file_name: Path
    mlflow_uri: str

In [22]:
from ML_MLOps.constants import *
from ML_MLOps.utils.common import read_yaml, create_directories, save_json

In [41]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)

            create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
          config = self.config.model_evaluation
          params = self.params.ElasticNet

          create_directories([config.root_dir])
          
          model_evaluation_config = ModelEvaluationConfig(
                root_dir=config.root_dir,
                test_X_path=config.test_X_path,
                test_y_path=config.test_y_path,
                model_path=config.model_path,
                metric_file_name=config.metric_file_name,
                all_params=params,
                mlflow_uri="https://dagshub.com/mjdocevedo/ML_MLOps.mlflow",
          )

          return model_evaluation_config

In [42]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [43]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        X_test = pd.read_csv(self.config.test_X_path)
        y_test = pd.read_csv(self.config.test_y_path)
        model = joblib.load(self.config.model_path)

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(X_test)

            (rmse, mae, r2) = self.eval_metrics(y_test, predicted_qualities)

            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case.

                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")

            else:
                mlflow.sklearn.log_model(model, "model")         


In [46]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config = model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-06-17 18:42:23,180: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-17 18:42:23,188: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-17 18:42:23,191: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-17 18:42:23,194: INFO: common: created directory at: artifacts]
[2024-06-17 18:42:23,197: INFO: common: created directory at: artifacts/model_evaluation]
[2024-06-17 18:42:23,871: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2024/06/17 18:42:36 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 4
Created version '4' of model 'ElasticnetModel'.
